<a href="https://colab.research.google.com/github/MahimaMaurya/Indigo/blob/main/Notebooks/3_GPT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install datasets
!pip install nltk
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9144ec59bf51b5efad359ce61f7fe6cb21c6120bee1f1ac6b935115dee5af528
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
import logging
import warnings
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score


In [14]:
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [15]:
df = pd.read_csv('/content/tokenized_output.csv', on_bad_lines='skip')
df1 = pd.read_csv('/content/test_data.csv', on_bad_lines='skip')
df.head()

,question,tokenized_question,stemmed_question,answer,tokenized_answer,stemmed_answer,question_length,answer_length
0,How can a piece of paper folded 103 times be l...,"['how', 'can', 'a', 'piece', 'of', 'paper', 'f...","['how', 'can', 'a', 'piec', 'of', 'paper', 'fo...",It isn’t. It just has a stack height longer t...,"['it', 'isn’t', 'it', 'just', 'has', 'a', 'sta...","['it', 'isn’t', 'it', 'just', 'ha', 'a', 'stac...",81,119
1,Did a landlord ever offer to lower your rent s...,"['did', 'a', 'landlord', 'ever', 'offer', 'to'...","['did', 'a', 'landlord', 'ever', 'offer', 'to'...",Did a landlord ever offer to lower your rent s...,"['did', 'a', 'landlord', 'ever', 'offer', 'to'...","['did', 'a', 'landlord', 'ever', 'offer', 'to'...",72,258
2,"If the Von Karman line is 62 miles above, then...","['if', 'the', 'von', 'karman', 'line', 'is', '...","['if', 'the', 'von', 'karman', 'line', 'is', '...",No. The Karman line has nothing to do with the...,"['no', 'the', 'karman', 'line', 'has', 'nothin...","['no', 'the', 'karman', 'line', 'ha', 'noth', ...",114,759
3,Do soldiers pick up fallen enemy weapons to us...,"['do', 'soldiers', 'pick', 'up', 'fallen', 'en...","['do', 'soldier', 'pick', 'up', 'fallen', 'ene...","It’s against the rules of war so no, if they r...","['it’s', 'against', 'the', 'rules', 'of', 'war...","['it’', 'against', 'the', 'rule', 'of', 'war',...",89,379
4,How do I get my son to stop making music?,"['how', 'do', 'i', 'get', 'my', 'son', 'to', '...","['how', 'do', 'i', 'get', 'my', 'son', 'to', '...","Wha- if he loves making music, let him make mu...","['wha', 'if', 'he', 'loves', 'making', 'music'...","['wha', 'if', 'he', 'love', 'make', 'music', '...",41,244


In [18]:
def prepare_data(data, file_path):
    df = pd.read_csv(file_path, on_bad_lines='skip')
    return df  # Or any other processed data

# Now you can call it:
train_data = prepare_data(df, '/content/tokenized_output.csv')
test_data = prepare_data(df1, '/content/test_data.csv')

In [29]:
from datasets import Dataset

In [ ]:
def create_text_dataset(file_path, tokenizer, block_size=512, overwrite_cache=True):
  """
  Creates a dataset for language modeling from a text file.

  Args:
    file_path: Path to the text file containing the data.
    tokenizer: The tokenizer to use for preprocessing.
    block_size: The maximum sequence length for each training example.
    overwrite_cache: Whether to overwrite the cached dataset.

  Returns:
    A dataset ready for training.
  """
    # Load the data
  df = pd.read_csv(file_path, on_bad_lines='skip')

  # Check the column names and adjust accordingly
  print(df.columns)  # Print the column names to verify
  text_column_name = 'tokenized_answer'  # Replace with the actual column name if different
  texts = df[text_column_name].tolist()

   # Add a padding token to the tokenizer
  tokenizer.pad_token = tokenizer.eos_token

  # Tokenize the texts and create a TextDataset
  encodings = tokenizer(texts, truncation=True, padding=True, max_length=block_size)
  dataset = Dataset.from_dict(encodings)

  return dataset

  # Implement dataset creation logic (e.g., using Hugging Face Datasets)
  pass

def train_language_model(model, tokenizer, data, output_dir, num_epochs=2, batch_size=16):
  """
  Trains a language model on the given dataset.

  Args:
    model: The pre-trained language model.
    tokenizer: The tokenizer used for preprocessing.
    data: The training dataset.
    output_dir: The directory to save the trained model and tokenizer.
    num_epochs: The number of training epochs.
    batch_size: The batch size for training.
  """

  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

  training_args = TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=True,
      num_train_epochs=num_epochs,
      per_device_train_batch_size=batch_size,
      save_steps=10_000,
      save_total_limit=2,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=data
  )

  trainer.train()
  trainer.save_model(output_dir)
  tokenizer.save_pretrained(output_dir)

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Create dataset
dataset_path = "/content/tokenized_output.csv"
dataset = create_text_dataset(dataset_path, tokenizer)

# Train the model
output_dir = "/content/fine_tuned_gpt_model"
train_language_model(model, tokenizer, dataset, output_dir)


Index(['question', 'tokenized_question', 'stemmed_question', 'answer',
       'tokenized_answer', 'stemmed_answer', 'question_length',
       'answer_length'],
      dtype='object')


In [3]:
!pip install torch
import torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [6]:
qa_model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
question_answering_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

In [10]:
import pandas as pd
test = pd.read_csv("/content/test_data.csv")

In [14]:
!pip install rouge-score
!pip install nltk
!pip install tqdm
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

In [15]:
def predict_answer_using_question_answering_model(question_answering_model, tokenizer, question, device="cuda"):
  """
  Generates an answer to a given question using a pre-trained question answering model.

  Args:
      question_answering_model: The loaded question answering model.
      tokenizer: The tokenizer used to preprocess text for the model.
      question: The user-provided question string.
      device: The device to use for computation ("cuda" for GPU, "cpu" otherwise).

  Returns:
      The predicted answer string, or an empty string if no answer is found.
  """

  inputs = tokenizer.encode_plus(question, return_tensors="pt").to(device)
  with torch.no_grad():
    outputs = question_answering_model(**inputs)

    # Handle cases where no answer is found
    if outputs.start_logits.max().item() < 0.0:
      return ""

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
  return answer

def evaluate_answer_predictions(ground_truth_answers, generated_answers):
  """
  Computes various evaluation metrics for a set of predicted answers to questions.

  Args:
      ground_truth_answers: A list of reference answer strings.
      generated_answers: A list of predicted answer strings.

  Returns:
      A tuple containing average BLEU, ROUGE-L, and F1 scores.
  """

  bleu_scores = []
  rouge_l_scores = []
  f1_scores = []

  rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

  for ref, pred in zip(ground_truth_answers, generated_answers):
    # Handle cases where no answer is predicted
    if not pred:
      continue

    # BLEU score
    bleu_score = sentence_bleu([ref.split()], pred.split())
    bleu_scores.append(bleu_score)

    # ROUGE-L score
    rouge_l_score = rouge.score(ref, pred)['rougeL'].fmeasure
    rouge_l_scores.append(rouge_l_score)

    # F1-score
    ref_tokens = set(ref.split())
    pred_tokens = set(pred.split())
    common_tokens = ref_tokens.intersection(pred_tokens)

    if len(common_tokens) == 0:
      f1 = 0.0
    else:
      precision = len(common_tokens) / len(pred_tokens)
      recall = len(common_tokens) / len(ref_tokens)
      f1 = 2 * (precision * recall) / (precision + recall)
    f1_scores.append(f1)

  if not bleu_scores:
    avg_bleu = 0.0
  else:
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

  avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
  avg_f1 = sum(f1_scores) / len(f1_scores)

  return avg_bleu, avg_rouge_l, avg_f1

# Example usage
device = "cuda" if torch.cuda.is_available() else "cpu"
question_answering_model.to(device)

questions = test.question
ground_truth_answers = test.answer

predicted_answers = [predict_answer_using_question_answering_model(question_answering_model, tokenizer, q, device) for q in questions]

avg_bleu, avg_rouge_l, avg_f1 = evaluate_answer_predictions(ground_truth_answers, predicted_answers)

print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-L Score: {avg_rouge_l}")
print(f"Average F1 Score: {avg_f1}")


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 0.00019881173839742889
Average ROUGE-L Score: 0.04767386358127768
Average F1 Score: 0.04163202635054907
